# Librerias

In [29]:
import pandas as pd
from pandas import json_normalize
import json
import numpy as np
import psycopg2
import pymysql
from sqlalchemy import create_engine
import time

## Extract

In [11]:
# Dataframe de todas las playlists
data = json.load(open("mpd.slice.1000-1999.json"))
df_start = json_normalize(data['playlists'])
df_start

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,disney,false,1000,1457827200,189,16,1,"[{'pos': 0, 'artist_name': 'Original Broadway ...",4,31428282,65,NaN
1,Indie Electro,false,1001,1417824000,165,18,2,"[{'pos': 0, 'artist_name': 'The Octopus Projec...",2,38241566,8,NaN
2,jack & jack,false,1002,1465430400,17,14,1,"[{'pos': 0, 'artist_name': 'Jack & Jack', 'tra...",3,3549358,3,NaN
3,vibes,false,1003,1498435200,225,195,2,"[{'pos': 0, 'artist_name': 'LANY', 'track_uri'...",91,51242585,157,NaN
4,Indie,false,1004,1498608000,165,118,1,"[{'pos': 0, 'artist_name': 'Youth Lagoon', 'tr...",74,42601098,92,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,woo,false,1995,1509062400,146,68,2,"[{'pos': 0, 'artist_name': 'blink-182', 'track...",13,29183178,24,NaN
996,NEW YEARS,false,1996,1420070400,38,37,1,"[{'pos': 0, 'artist_name': 'Big Sean', 'track_...",3,9354755,34,NaN
997,JESUS,false,1997,1508371200,40,28,2,"[{'pos': 0, 'artist_name': 'Elevation Worship'...",8,10940019,22,NaN
998,yep,false,1998,1448236800,29,27,1,"[{'pos': 0, 'artist_name': 'Oasis', 'track_uri...",3,7260645,20,NaN


In [12]:
# Dataframe solamente de las canciones
df_tracks = pd.DataFrame(df_start.tracks[0])
df_tracks["sk_playlist"] = df_start.pid[0]

for i in range(1, len(df_start)):
    act = pd.DataFrame(df_start.tracks[i])
    act["sk_playlist"] = df_start.pid[i]
    
    df_tracks = pd.concat([df_tracks,act])
df_tracks

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,sk_playlist
0,0,Original Broadway Cast - The Little Mermaid,spotify:track:5IbCV9Icebx8rR6wAp5hhP,spotify:artist:3TymzPhJTMyupk7P5xkahM,Fathoms Below - Broadway Cast Recording,spotify:album:3ULJeOMgroG27dpn27MDfS,154506,The Little Mermaid: Original Broadway Cast Rec...,1000
1,1,Original Broadway Cast - The Little Mermaid,spotify:track:6rKVAvjHcxAzZ1BHtwh5yC,spotify:artist:3TymzPhJTMyupk7P5xkahM,Daughters Of Triton - Broadway Cast Recording,spotify:album:3ULJeOMgroG27dpn27MDfS,79066,The Little Mermaid: Original Broadway Cast Rec...,1000
2,2,Original Broadway Cast - The Little Mermaid,spotify:track:6Jlkb1Wh08RYHstWScsTvg,spotify:artist:3TymzPhJTMyupk7P5xkahM,The World Above - Broadway Cast Recording,spotify:album:3ULJeOMgroG27dpn27MDfS,94600,The Little Mermaid: Original Broadway Cast Rec...,1000
3,3,Original Broadway Cast - The Little Mermaid,spotify:track:0XhC8bfStML9ygBmfOt1JJ,spotify:artist:3TymzPhJTMyupk7P5xkahM,Human Stuff - Broadway Cast Recording,spotify:album:3ULJeOMgroG27dpn27MDfS,151480,The Little Mermaid: Original Broadway Cast Rec...,1000
4,4,Original Broadway Cast - The Little Mermaid,spotify:track:0ABxAcsRWlqckkyONsfP67,spotify:artist:3TymzPhJTMyupk7P5xkahM,I Want the Good Times Back - Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,297920,The Little Mermaid: Original Broadway Cast Rec...,1000
...,...,...,...,...,...,...,...,...,...
11,11,Seether,spotify:track:2G8gLafzmoHwa6RBih74J3,spotify:artist:6B5c4sch27tWHAGdarpPaW,Broken - Featuring Amy Lee,spotify:album:5qHjA0aU9vt9c2UwDeEGjS,258333,Disclaimer II,1999
12,12,Three Days Grace,spotify:track:1bPRhOz3nOfEIoLjB1YYT3,spotify:artist:2xiIXseIJcq3nG7C8fHeBj,No More,spotify:album:0vv1zKShlm7WuawEup5Mf4,225493,Life Starts Now,1999
13,13,Three Days Grace,spotify:track:1o4Pf2GIMDCD7ifIM2yI77,spotify:artist:2xiIXseIJcq3nG7C8fHeBj,Last to Know,spotify:album:0vv1zKShlm7WuawEup5Mf4,207466,Life Starts Now,1999
14,14,3 Doors Down,spotify:track:0nNVR2iDM3eVzEgMi78vQm,spotify:artist:2RTUTCvo6onsAnheUk3aL9,Let Me Go,spotify:album:44AVPs6VGKnZHinQNWd6Xd,243053,The Greatest Hits,1999


In [13]:
# ids sk_track
id_tracks = df_tracks["track_uri"].unique()
id_tracks = pd.DataFrame(id_tracks, columns=["track_uri"]).reset_index()
id_tracks = id_tracks.rename({"index":"sk_track"},axis=1)

# ids sk_album
id_album = df_tracks["album_uri"].unique()
id_album = pd.DataFrame(id_album, columns=["album_uri"]).reset_index()
id_album = id_album.rename({"index":"sk_album"},axis=1)

# ids sk_artists
id_artist = df_tracks["artist_uri"].unique()
id_artist  = pd.DataFrame(id_artist, columns=["artist_uri"]).reset_index()
id_artist = id_artist.rename({"index":"sk_artist"},axis=1)

In [14]:
# creacion del dataframe tracks con sus sk_id's y demas identificadores
df_tracks = df_tracks.merge(id_tracks, left_on="track_uri", right_on="track_uri", how="inner")
df_tracks = df_tracks.merge(id_album, left_on="album_uri", right_on="album_uri", how="inner")
df_tracks = df_tracks.merge(id_artist, left_on="artist_uri", right_on="artist_uri", how="inner")

sk_playlist_ser = pd.Series(df_tracks['sk_playlist'].drop_duplicates())
df_start = df_start.assign(sk_playlist=sk_playlist_ser.values)
df_tracks["sk_playlist_track"] = range(0, len(df_tracks))
df_tracks

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,sk_playlist,sk_track,sk_album,sk_artist,sk_playlist_track
0,0,Original Broadway Cast - The Little Mermaid,spotify:track:5IbCV9Icebx8rR6wAp5hhP,spotify:artist:3TymzPhJTMyupk7P5xkahM,Fathoms Below - Broadway Cast Recording,spotify:album:3ULJeOMgroG27dpn27MDfS,154506,The Little Mermaid: Original Broadway Cast Rec...,1000,0,0,0,0
1,1,Original Broadway Cast - The Little Mermaid,spotify:track:6rKVAvjHcxAzZ1BHtwh5yC,spotify:artist:3TymzPhJTMyupk7P5xkahM,Daughters Of Triton - Broadway Cast Recording,spotify:album:3ULJeOMgroG27dpn27MDfS,79066,The Little Mermaid: Original Broadway Cast Rec...,1000,1,0,0,1
2,2,Original Broadway Cast - The Little Mermaid,spotify:track:6Jlkb1Wh08RYHstWScsTvg,spotify:artist:3TymzPhJTMyupk7P5xkahM,The World Above - Broadway Cast Recording,spotify:album:3ULJeOMgroG27dpn27MDfS,94600,The Little Mermaid: Original Broadway Cast Rec...,1000,2,0,0,2
3,3,Original Broadway Cast - The Little Mermaid,spotify:track:0XhC8bfStML9ygBmfOt1JJ,spotify:artist:3TymzPhJTMyupk7P5xkahM,Human Stuff - Broadway Cast Recording,spotify:album:3ULJeOMgroG27dpn27MDfS,151480,The Little Mermaid: Original Broadway Cast Rec...,1000,3,0,0,3
4,4,Original Broadway Cast - The Little Mermaid,spotify:track:0ABxAcsRWlqckkyONsfP67,spotify:artist:3TymzPhJTMyupk7P5xkahM,I Want the Good Times Back - Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,297920,The Little Mermaid: Original Broadway Cast Rec...,1000,4,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66617,137,Masked Intruder,spotify:track:3ug7AfV5SUyqFHqEk3rtS3,spotify:artist:4qUFkkArfii3qNG8TnbPfc,Stick 'Em Up,spotify:album:2rcd8wxONSQDKmYdut3iRa,109586,Masked Intruder,1995,36140,20193,10299,66617
66618,145,Masked Intruder,spotify:track:2bziYzO5MmVCPnoCqreabt,spotify:artist:4qUFkkArfii3qNG8TnbPfc,I Don't Wanna Say Goodbye to You Tonight,spotify:album:45SR5DEOA0uhHC975PWhA8,113546,M.I.,1995,36143,20196,10299,66618
66619,22,The Cataracs,spotify:track:0TDD00m2Mypa10QOBbb34h,spotify:artist:7C64wNX3howEFZjAYRKsfP,All You,spotify:album:4e3OFoysTjZ4XSK2aAER3c,214786,Gordo Taqueria,1996,36146,20198,10300,66619
66620,35,Young T.H.U.G.,spotify:track:0VL32DjEZZiOj2dYo9a0NX,spotify:artist:6pxwzi0xDny2rMI1KJFZoi,Danny Glover,spotify:album:664vzbSX8o7l1fmXsiI7HE,197528,Trap Still Boomin' 2,1996,36149,20201,10301,66620


## Transform

In [15]:
# d_albums
d_album = df_tracks[["sk_album", "album_uri", "album_name"]].drop_duplicates()
print(d_album.shape)

# d_artists 
d_artist = df_tracks[["sk_artist", "artist_uri", "artist_name"]].drop_duplicates()
print(d_artist.shape)

# d_playlists
d_playlist = df_start[["sk_playlist", "pid", "name", "collaborative", "modified_at", "num_followers", "num_edits"]].drop_duplicates()
print(d_playlist.shape)

# d_tracks
d_track = df_tracks[["sk_track", "track_uri", "track_name", "duration_ms"]].drop_duplicates()
print(d_track.shape)

# d_playlist_tracks
d_playlist_tracks = df_tracks[[
    "sk_playlist_track", 
    "sk_playlist",  
    "sk_artist",
    "sk_album", 
    "sk_track",
    "pos"
]].drop_duplicates()
print(d_playlist_tracks.shape)


(20212, 3)
(10303, 3)
(1000, 7)
(36175, 4)
(66622, 6)


## Load

In [19]:
# conexion con servidor postgres. Use un contenedor de docker para conectarme a la base de datos
conn = psycopg2.connect(
    database = "postgres",
    user = "postgres",
    host = "localhost", 
    password = "daniel123", 
    port = "5432"
    )

### d_artists

In [20]:
q_dart = [tuple(x) for x in d_artist.to_numpy()]
q_dar = "INSERT INTO " + "d_artists" + "(" + "sk_id, bk_id, name" + ")" + "VALUES(" + "%s, %s, %s" + ");"
cursor = conn.cursor()

try: 
    cursor.executemany(q_dar, q_dart)
    conn.commit()
    print("Data loaded successfully...")

except (Exception, psycopg2.DatabaseError) as error:
    print("Error: ", error)
    conn.rollback()
    print("Ups, something went wrong...")

cursor.close()

Data loaded successfully...


### d_albums

In [21]:
q_dalt = [tuple(x) for x in d_album.to_numpy()]
q_dal = "INSERT INTO " + "d_albums" + "(" + "sk_id, bk_id, name" + ")" + "VALUES(" + "%s, %s, %s" + ");"
cursor = conn.cursor()

try: 
    cursor.executemany(q_dal, q_dalt)
    conn.commit()
    print("Data loaded successfully...")

except (Exception, psycopg2.DatabaseError) as error:
    print("Error: ", error)
    conn.rollback()
    print("Ups, something went wrong...")

cursor.close()

Data loaded successfully...


### d_tracks

In [22]:
q_trat = [tuple(x) for x in d_track.to_numpy()]
q_tra = "INSERT INTO " + "d_tracks" + "(" + "sk_id, bk_id, name, duration" + ")" + "VALUES(" + "%s, %s, %s, %s" + ");"
cursor = conn.cursor()

try: 
    cursor.executemany(q_tra, q_trat)
    conn.commit()
    print("Data loaded successfully...")

except (Exception, psycopg2.DatabaseError) as error:
    print("Error: ", error)
    conn.rollback()
    print("Ups, something went wrong...")

cursor.close()

Data loaded successfully...


### d_playlists

In [24]:
q_plat = [tuple(x) for x in d_playlist.to_numpy()]
q_pla = ("INSERT INTO " + "d_playlists" + "(" + "sk_id, bk_id, name, collaborative, last_modified_at, num_followers, num_edits" + ")" 
         + "VALUES(" + "%s, %s, %s, %s, to_timestamp(%s), %s, %s" + ");")
cursor = conn.cursor()


try: 
    cursor.executemany(q_pla, q_plat)
    conn.commit()
    print("Data loaded successfully...")

except (Exception, psycopg2.DatabaseError) as error:
    print("Error: ", error)
    conn.rollback()
    print("Ups, something went wrong...")

cursor.close()

Data loaded successfully...


### f_playlist_tracks

In [25]:
d_playlist_tracks = d_playlist_tracks.convert_dtypes()
q_ptt = [tuple(x) for x in d_playlist_tracks.to_numpy()]

q_pt = ("INSERT INTO " + "f_playlist_tracks" + "(" + "id, sk_playlist, sk_artist, sk_album, sk_track, track_position"
          + ")" + "VALUES(" + "%s, %s, %s, %s, %s, %s" + ");")

cursor = conn.cursor()

try: 
    cursor.executemany(q_pt, q_ptt)
    conn.commit()
    print("Data loaded successfully...")

except (Exception, psycopg2.DatabaseError) as error:
    print("Error: ", error)
    conn.rollback()
    print("Ups, something went wrong...")

cursor.close()

La carga de datos fue un exito...
